In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## No data from september 1980(GOES03) to April 1983; From may 1983(GOES06)¶  
Solar Cycle 20: May 1964 to March 1976  
Solar Cycle 21: March 1976 to October 1986  
Solar Cycle 22: October 1986 to June 1996  
Solar Cycle 23: June 1996 to December 2008  
Solar Cycle 24: December 2008 to December 2019  
Solar Cycle 25: 2019 - (projected) 2030  

In [3]:
import os
import csv
import json
import shutil
import requests
import pandas as pd
import urllib.request
from typing import Tuple

DATA_DIR = "data_from_sol_22_updated"
missing_infos = {}

def create_folder(year: int):
    #create folder to hold the data
    os.makedirs(f"{DATA_DIR}/{year}", exist_ok = True)

def download_file(year: int, url: str, file_name: str) -> None:
    # download the file from `url` and save it locally under `file_name`:
    with urllib.request.urlopen(url) as response, open(f"{DATA_DIR}/{year}/{file_name}", 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

def get_continuous_ranges(nums: list) -> list[Tuple]:
    # helper function to get continuous interval
    nums = sorted(set(nums))
    gaps = [[s, e] for s, e in zip(nums, nums[1:]) if s+1 < e]
    edges = iter(nums[:1] + sum(gaps, []) + nums[-1:])
    return list(zip(edges, edges))
    
def get_missing_info(file_path: str, found_data_key: str) -> Tuple[list[str], list[int]]:
    # get different missing data informations
    list_missing_intervals = []
    list_missing_length = []
    
    df = pd.read_csv(file_path)
    if found_data_key == "starting_point_new":
        col = "A_AVG"
    elif found_data_key == "starting_point_old" or found_data_key == "starting_point_old_v2":
        col = "xl"
        
    df_missing_values = df.loc[df[col] == -99999]
    missing_indexes = df_missing_values.index.tolist()

    intervals = get_continuous_ranges(missing_indexes)
    for interval in intervals:
        st =  df.iloc[[interval[0]]].values.tolist()[0][0]
        end = df.iloc[[interval[1]]].values.tolist()[0][0]
        list_missing_intervals.append(f"{st} to {end}")
        list_missing_length.append(interval[1] - interval[0] + 1)
    
    return list_missing_intervals, list_missing_length

def pre_process_data(missing_infos:dict, satelite: str, year: int, month: str, file_name: str) -> dict:
    """
    Extract the important rows and save them as csv
    Also get some information about missing values
    """
    data_dict = {}
    yearly_missing_dict = {}
    missing_row_counter = 0
    
    csv_filename_with_dir = f"{DATA_DIR}/{year}/{file_name}"
    starting_point_found = False
    found_data_key = ""
    original_data_dict = {
        "starting_point_new" : ['time_tag', 'A_QUAL_FLAG', 'A_NUM_PTS', 'A_AVG', 'B_QUAL_FLAG', 'B_NUM_PTS', 'B_AVG'],
        "starting_point_old" : ['time_tag', 'xl', 'xs'],
        "starting_point_old_v2" : ['time_tag', 'xs', 'xl'],
    }
    
    with open(csv_filename_with_dir, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in reader:
            if row == original_data_dict["starting_point_new"]:
                starting_point_found = True
                found_data_key = "starting_point_new"
                for col in original_data_dict[found_data_key]:
                    data_dict[col] = []
                continue
                
            if row == original_data_dict["starting_point_old"]:
                starting_point_found = True
                found_data_key = "starting_point_old"
                for col in original_data_dict[found_data_key]:
                    data_dict[col] = []
                continue
            if row == original_data_dict["starting_point_old_v2"]:
                starting_point_found = True
                found_data_key = "starting_point_old_v2"
                for col in original_data_dict[found_data_key]:
                    data_dict[col] = []
                continue
                
            if starting_point_found:
                for i, col in enumerate(original_data_dict[found_data_key]):
                    data_dict[col].append(row[i])

    df = pd.DataFrame.from_dict(data_dict)
    df.to_csv(csv_filename_with_dir, index = False)
    
    yearly_missing_dict["file_name"] = f"{year}/{file_name}"
    yearly_missing_dict["number of total rows"] = df.shape[0]
    list_missing_intervals, list_missing_length = get_missing_info(csv_filename_with_dir, found_data_key)
    yearly_missing_dict["number of missing rows"] = sum(list_missing_length)
    yearly_missing_dict["missing rows each interval"] = list_missing_length
    yearly_missing_dict["missing intervals"] = list_missing_intervals
    missing_infos[f"{satelite}_{month}"] = yearly_missing_dict
    
    return missing_infos
    
def download_data_by_year():
    """
    Need some steps to get the data from www.ncei.noaa.gov
        -> Need date range of the year by month
        -> Need the filename
        -> Need the URL for file in NOAA website
        -> Download the data
    """
    df = pd.read_csv("existing_data_history.csv")
    years = [year for year in range(1986, 2021)]
    
    for year in years:
        print(f"Downloading data for {year}")
        missing_infos = {}
        #create folder to hold data
        create_folder(year)
        
        filtered_df = df[df["year"] == year]
        rows_as_list = filtered_df.values.tolist()
        for row in rows_as_list:
            satelite, year, month, file_name, url = row
            month = f"{month:02d}"
            download_file(year, url, file_name)
            print(f"Download completed of {file_name}, url: {url}")
            
            missing_infos = pre_process_data(missing_infos, satelite, year, month, file_name)
            print(f"Pre-processing completed of {file_name}")
            print("......")
    
        print(f"saving summary for year {year}")
        with open(f"{DATA_DIR}/{year}_summary.json", "w") as fp:
            json.dump(missing_infos, fp)
        
        print(f"--------End of {year}--------")

download_data_by_year()

Download completed of g05_xrs_1m_19860101_19860131.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1986/01/goes05/csv/g05_xrs_1m_19860101_19860131.csv
Pre-processing completed of g05_xrs_1m_19860101_19860131.csv
......
Download completed of g06_xrs_1m_19860101_19860131.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1986/01/goes06/csv/g06_xrs_1m_19860101_19860131.csv
Pre-processing completed of g06_xrs_1m_19860101_19860131.csv
......
Download completed of g05_xrs_1m_19860201_19860228.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1986/02/goes05/csv/g05_xrs_1m_19860201_19860228.csv
Pre-processing completed of g05_xrs_1m_19860201_19860228.csv
......
Download completed of g06_xrs_1m_19860201_19860228.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1986/02/goes06/csv/g06_xrs_1m_19860201_19860228.csv
Pre-processing completed of g06_xrs_1m_19860201_

Download completed of g06_xrs_1m_19870501_19870531.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1987/05/goes06/csv/g06_xrs_1m_19870501_19870531.csv
Pre-processing completed of g06_xrs_1m_19870501_19870531.csv
......
Download completed of g07_xrs_1m_19870501_19870531.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1987/05/goes07/csv/g07_xrs_1m_19870501_19870531.csv
Pre-processing completed of g07_xrs_1m_19870501_19870531.csv
......
Download completed of g06_xrs_1m_19870601_19870630.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1987/06/goes06/csv/g06_xrs_1m_19870601_19870630.csv
Pre-processing completed of g06_xrs_1m_19870601_19870630.csv
......
Download completed of g07_xrs_1m_19870601_19870630.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1987/06/goes07/csv/g07_xrs_1m_19870601_19870630.csv
Pre-processing completed of g07_xrs_1m_19870601_

Download completed of g06_xrs_1m_19880901_19880930.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1988/09/goes06/csv/g06_xrs_1m_19880901_19880930.csv
Pre-processing completed of g06_xrs_1m_19880901_19880930.csv
......
Download completed of g07_xrs_1m_19880901_19880930.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1988/09/goes07/csv/g07_xrs_1m_19880901_19880930.csv
Pre-processing completed of g07_xrs_1m_19880901_19880930.csv
......
Download completed of g06_xrs_1m_19881001_19881031.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1988/10/goes06/csv/g06_xrs_1m_19881001_19881031.csv
Pre-processing completed of g06_xrs_1m_19881001_19881031.csv
......
Download completed of g07_xrs_1m_19881001_19881031.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1988/10/goes07/csv/g07_xrs_1m_19881001_19881031.csv
Pre-processing completed of g07_xrs_1m_19881001_

Download completed of g06_xrs_1m_19900101_19900131.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1990/01/goes06/csv/g06_xrs_1m_19900101_19900131.csv
Pre-processing completed of g06_xrs_1m_19900101_19900131.csv
......
Download completed of g07_xrs_1m_19900101_19900131.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1990/01/goes07/csv/g07_xrs_1m_19900101_19900131.csv
Pre-processing completed of g07_xrs_1m_19900101_19900131.csv
......
Download completed of g06_xrs_1m_19900201_19900228.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1990/02/goes06/csv/g06_xrs_1m_19900201_19900228.csv
Pre-processing completed of g06_xrs_1m_19900201_19900228.csv
......
Download completed of g07_xrs_1m_19900201_19900228.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1990/02/goes07/csv/g07_xrs_1m_19900201_19900228.csv
Pre-processing completed of g07_xrs_1m_19900201_

Download completed of g06_xrs_1m_19910501_19910531.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1991/05/goes06/csv/g06_xrs_1m_19910501_19910531.csv
Pre-processing completed of g06_xrs_1m_19910501_19910531.csv
......
Download completed of g07_xrs_1m_19910501_19910531.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1991/05/goes07/csv/g07_xrs_1m_19910501_19910531.csv
Pre-processing completed of g07_xrs_1m_19910501_19910531.csv
......
Download completed of g06_xrs_1m_19910601_19910630.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1991/06/goes06/csv/g06_xrs_1m_19910601_19910630.csv
Pre-processing completed of g06_xrs_1m_19910601_19910630.csv
......
Download completed of g07_xrs_1m_19910601_19910630.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/1991/06/goes07/csv/g07_xrs_1m_19910601_19910630.csv
Pre-processing completed of g07_xrs_1m_19910601_

...

Pre-processing completed of g15_xrs_1m_20141001_20141031.csv
......
Download completed of g15_xrs_1m_20141101_20141130.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2014/11/goes15/csv/g15_xrs_1m_20141101_20141130.csv
Pre-processing completed of g15_xrs_1m_20141101_20141130.csv
......
Download completed of g15_xrs_1m_20141201_20141231.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2014/12/goes15/csv/g15_xrs_1m_20141201_20141231.csv
Pre-processing completed of g15_xrs_1m_20141201_20141231.csv
......
saving summary for year 2014
--------End of 2014--------
Download completed of g13_xrs_1m_20150101_20150131.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2015/01/goes13/csv/g13_xrs_1m_20150101_20150131.csv
Pre-processing completed of g13_xrs_1m_20150101_20150131.csv
......
Download completed of g15_xrs_1m_20150101_20150131.csv, url:  https://www.ncei.noaa.gov/data/goes-space-enviro

Download completed of g13_xrs_1m_20160301_20160331.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2016/03/goes13/csv/g13_xrs_1m_20160301_20160331.csv
Pre-processing completed of g13_xrs_1m_20160301_20160331.csv
......
Download completed of g15_xrs_1m_20160301_20160331.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2016/03/goes15/csv/g15_xrs_1m_20160301_20160331.csv
Pre-processing completed of g15_xrs_1m_20160301_20160331.csv
......
Download completed of g13_xrs_1m_20160401_20160430.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2016/04/goes13/csv/g13_xrs_1m_20160401_20160430.csv
Pre-processing completed of g13_xrs_1m_20160401_20160430.csv
......
Download completed of g14_xrs_1m_20160401_20160430.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2016/04/goes14/csv/g14_xrs_1m_20160401_20160430.csv
Pre-processing completed of g14_xrs_1m_20160401_

Download completed of g13_xrs_1m_20170301_20170331.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2017/03/goes13/csv/g13_xrs_1m_20170301_20170331.csv
Pre-processing completed of g13_xrs_1m_20170301_20170331.csv
......
Download completed of g15_xrs_1m_20170301_20170331.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2017/03/goes15/csv/g15_xrs_1m_20170301_20170331.csv
Pre-processing completed of g15_xrs_1m_20170301_20170331.csv
......
Download completed of g13_xrs_1m_20170401_20170430.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2017/04/goes13/csv/g13_xrs_1m_20170401_20170430.csv
Pre-processing completed of g13_xrs_1m_20170401_20170430.csv
......
Download completed of g15_xrs_1m_20170401_20170430.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2017/04/goes15/csv/g15_xrs_1m_20170401_20170430.csv
Pre-processing completed of g15_xrs_1m_20170401_

Download completed of g14_xrs_1m_20180601_20180630.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2018/06/goes14/csv/g14_xrs_1m_20180601_20180630.csv
Pre-processing completed of g14_xrs_1m_20180601_20180630.csv
......
Download completed of g15_xrs_1m_20180601_20180630.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2018/06/goes15/csv/g15_xrs_1m_20180601_20180630.csv
Pre-processing completed of g15_xrs_1m_20180601_20180630.csv
......
Download completed of g14_xrs_1m_20180701_20180731.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2018/07/goes14/csv/g14_xrs_1m_20180701_20180731.csv
Pre-processing completed of g14_xrs_1m_20180701_20180731.csv
......
Download completed of g15_xrs_1m_20180701_20180731.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2018/07/goes15/csv/g15_xrs_1m_20180701_20180731.csv
Pre-processing completed of g15_xrs_1m_20180701_

Download completed of g14_xrs_1m_20191001_20191031.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2019/10/goes14/csv/g14_xrs_1m_20191001_20191031.csv
Pre-processing completed of g14_xrs_1m_20191001_20191031.csv
......
Download completed of g15_xrs_1m_20191001_20191031.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2019/10/goes15/csv/g15_xrs_1m_20191001_20191031.csv
Pre-processing completed of g15_xrs_1m_20191001_20191031.csv
......
Download completed of g14_xrs_1m_20191101_20191130.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2019/11/goes14/csv/g14_xrs_1m_20191101_20191130.csv
Pre-processing completed of g14_xrs_1m_20191101_20191130.csv
......
Download completed of g15_xrs_1m_20191101_20191130.csv, url:  https://www.ncei.noaa.gov/data/goes-space-environment-monitor/access/avg/2019/11/goes15/csv/g15_xrs_1m_20191101_20191130.csv
Pre-processing completed of g15_xrs_1m_20191101_